In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
from datetime import datetime
import numpy as np
import ast 
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

individual_device_data = pd.read_csv(r'data/anon_processed_unique_device_v3.csv')
campaign_level_data = pd.read_csv(r'data/anan_campaign_modeling_data_v3.csv')

# ---------------------- Nearest Neighbor 
data_path = 'data/anan_campaign_modeling_data_v3.csv'  # Update with the correct path to your dataset
data = pd.read_csv(data_path)

# Normalize Genre Columns to Percentages
genre_columns = [
    'Classics', 'Comedy', 'Other', 'Reality', 'News and Information',
    'Drama', 'Action & Adventure', 'Thriller', 'Sci-Fi & Fantasy', 'Horror', 'Western',
    'Documentaries', 'Sports', 'Instructional & Educational', 'Home & Lifestyle', 'Romance',
    'Anime', 'Musical', 'Independent', 'Entertainment', 'Paranormal', 'Music', 'Gay & Lesbian',
    'Crime', 'Food & Cooking', 'Faith & Spirituality', 'Game Show', 'Dance', 'Children & Family',
    'Telenovela', 'Talk Show', 'Variety Show', 'War', 'Young Adult', 'None'
]

data['genre_sum'] = data[genre_columns].sum(axis=1)
data[genre_columns] = data[genre_columns].div(data['genre_sum'], axis=0).fillna(0)
data.drop(columns=['genre_sum'], inplace=True)

# Features for Nearest Neighbors
X = data[['impressions', 'clicks'] + genre_columns]

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train Nearest Neighbors Model
nn_model = NearestNeighbors(n_neighbors=1, metric='euclidean')
nn_model.fit(X_scaled)

# Save the Model and Scaler
joblib.dump(nn_model, 'models/nearest_neighbors_model.pkl')
joblib.dump(scaler, 'models/nearest_neighbors_scaler.pkl')
print("Nearest Neighbors model and scaler saved successfully!")

# ---------------------- RF Classifier 
# Load Dataset
data_path = 'data/anan_campaign_modeling_data_v3.csv'  # Update with the correct path to your dataset
data = pd.read_csv(data_path)

# Normalize Genre Columns to Percentages
genre_columns = [
    'Classics', 'Comedy', 'Other', 'Reality', 'News and Information',
    'Drama', 'Action & Adventure', 'Thriller', 'Sci-Fi & Fantasy', 'Horror', 'Western',
    'Documentaries', 'Sports', 'Instructional & Educational', 'Home & Lifestyle', 'Romance',
    'Anime', 'Musical', 'Independent', 'Entertainment', 'Paranormal', 'Music', 'Gay & Lesbian',
    'Crime', 'Food & Cooking', 'Faith & Spirituality', 'Game Show', 'Dance', 'Children & Family',
    'Telenovela', 'Talk Show', 'Variety Show', 'War', 'Young Adult', 'None'
]

data['genre_sum'] = data[genre_columns].sum(axis=1)
data[genre_columns] = data[genre_columns].div(data['genre_sum'], axis=0).fillna(0)
data.drop(columns=['genre_sum'], inplace=True)

# Encode 'score' Column
score_mapping = {'Poor': 0, 'Fair': 1, 'Good': 2, 'Excellent': 3}
data['score_encoded'] = data['score'].map(score_mapping)

# Feature Selection and Target
X = data[['impressions', 'clicks'] + genre_columns]
y = data['score_encoded']

# train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42, n_estimators=100)
rf_classifier.fit(X_train, y_train)

# tvaluate Model
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=score_mapping.keys()))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save Model
joblib.dump(rf_classifier, 'models/rf_classifier.pkl')
print("Random Forest Classifier saved as 'rf_classifier.pkl'")

# ---------------------- RF Regressor (Average Minutes)

# Load Dataset
data_path = 'data/anan_campaign_modeling_data_v3.csv'  
data = pd.read_csv(data_path)

# Normalize Genre Columns to Percentages
genre_columns = [
    'Classics', 'Comedy', 'Other', 'Reality', 'News and Information',
    'Drama', 'Action & Adventure', 'Thriller', 'Sci-Fi & Fantasy', 'Horror', 'Western',
    'Documentaries', 'Sports', 'Instructional & Educational', 'Home & Lifestyle', 'Romance',
    'Anime', 'Musical', 'Independent', 'Entertainment', 'Paranormal', 'Music', 'Gay & Lesbian',
    'Crime', 'Food & Cooking', 'Faith & Spirituality', 'Game Show', 'Dance', 'Children & Family',
    'Telenovela', 'Talk Show', 'Variety Show', 'War', 'Young Adult', 'None'
]

data['genre_sum'] = data[genre_columns].sum(axis=1)
data[genre_columns] = data[genre_columns].div(data['genre_sum'], axis=0).fillna(0)
data.drop(columns=['genre_sum'], inplace=True)

# Feature Selection and Target
X = data[['impressions', 'clicks'] + genre_columns]
y = data['avg_time_watched_per_device']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42, n_estimators=100)
rf_regressor.fit(X_train, y_train)

# Evaluate Model
y_pred = rf_regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Regression Model Performance:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

# Save Model
joblib.dump(rf_regressor, 'models/regression_model.pkl')
print("Random Forest Regressor saved as 'regression_model.pkl'")



Nearest Neighbors model and scaler saved successfully!
Accuracy: 0.56
Classification Report:
              precision    recall  f1-score   support

        Poor       0.25      0.33      0.29         3
        Fair       0.33      0.50      0.40         2
        Good       1.00      0.83      0.91         6
   Excellent       0.50      0.40      0.44         5

    accuracy                           0.56        16
   macro avg       0.52      0.52      0.51        16
weighted avg       0.62      0.56      0.58        16

Confusion Matrix:
[[1 1 0 1]
 [1 1 0 0]
 [0 0 5 1]
 [2 1 0 2]]
Random Forest Classifier saved as 'rf_classifier.pkl'
Regression Model Performance:
Mean Absolute Error (MAE): 24.16
Mean Squared Error (MSE): 1467.87
R^2 Score: -0.19
Random Forest Regressor saved as 'regression_model.pkl'


In [2]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.5.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License
        
        Copyright (c) 2007-2024 The scikit-learn developers.
        All rights reserved.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distribution.
        
        * Neither the name of the copyright holder nor the names of its
          contributors may be used to endorse or promote produ